## 1. Perkenalan

DS=Suma Bimantoro Andru

## 2.- Import Libraries

In [109]:
# Import Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler,OrdinalEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras import Model 
import tensorflow as tf
from tensorflow import keras
import pickle
import requests
import json
import requests

## 3 - Data Loading

In [110]:
#import data train
data1=pd.read_csv('Train-1542865627584.csv') #label #done
data2=pd.read_csv('Train_Beneficiarydata-1542865627584.csv')
data3=pd.read_csv('Train_Inpatientdata-1542865627584.csv')
data4=pd.read_csv('Train_Outpatientdata-1542865627584.csv')

In [111]:
#import data test
datates1=pd.read_csv('Test-1542969243754.csv') 
datates2=pd.read_csv('Test_Beneficiarydata-1542969243754.csv')
datates3=pd.read_csv('Test_Inpatientdata-1542969243754.csv')
datates4=pd.read_csv('Test_Outpatientdata-1542969243754.csv')

## EDA


## Feature Selection

In [112]:
#Pilih Feature yang berkorelasi dari Beneficiarydata
data2new=data2[['BeneID','Gender','RenalDiseaseIndicator','NoOfMonths_PartACov','NoOfMonths_PartBCov','ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis','ChronicCond_rheumatoidarthritis','ChronicCond_stroke','IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt']]
data2tesnew=datates2[['BeneID','Gender','RenalDiseaseIndicator','NoOfMonths_PartACov','NoOfMonths_PartBCov','ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis','ChronicCond_rheumatoidarthritis','ChronicCond_stroke','IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt']]

In [113]:
#Pilih Feature yang berkorelasi dari Inpatientdata
data3new=data3[['BeneID','Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]
data3tesnew=datates3[['BeneID','Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]

In [114]:
#Pilih Feature yang berkorelasi dari Outpatientdata
data4new=data4[['Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]
data4tesnew=datates4[['Provider','InscClaimAmtReimbursed','DeductibleAmtPaid']]

In [117]:
#merge 1 n 3 with Provider Foregn key
data1n3=pd.merge(data1,data3new,on='Provider',how='left')
datates1n3=pd.merge(datates1,data3tesnew,on='Provider',how='left')

In [118]:
#merge 1,3 and 2
data13n2=pd.merge(data2new,data1n3,on='BeneID',how='outer')
datates1n3=pd.merge(data2tesnew,datates1n3,on='BeneID',how='outer')

In [119]:
#merge 132 and 4
data132n4=pd.merge(data4new,data13n2,on='Provider',how='outer')
datatese132n4=pd.merge(data4tesnew,datates1n3,on='Provider',how='outer')

In [122]:
remove_columns=['BeneID']

final_train=data132n4.drop(columns=remove_columns, axis=1)
final_test=datatese132n4.drop(columns=remove_columns, axis=1)

In [124]:
final_train

,Provider,InscClaimAmtReimbursed_x,DeductibleAmtPaid_x,Gender,RenalDiseaseIndicator,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,...,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud,InscClaimAmtReimbursed_y,DeductibleAmtPaid_y
0,PRV56011,30.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN
1,PRV56011,40.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN
2,PRV56011,200.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN
3,PRV56011,20.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN
4,PRV56011,40.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11416113,PRV54054,NaN,NaN,2.0,Y,12.0,12.0,1.0,1.0,1.0,...,1.0,1.0,2.0,15000.0,2136.0,2810.0,560.0,No,5000.0,1068.0
11416114,PRV51136,NaN,NaN,1.0,0,12.0,12.0,2.0,2.0,2.0,...,2.0,2.0,2.0,20000.0,1068.0,400.0,220.0,No,20000.0,1068.0
11416115,PRV51977,NaN,NaN,1.0,Y,12.0,12.0,2.0,1.0,1.0,...,2.0,1.0,2.0,24420.0,3204.0,11670.0,3930.0,No,8000.0,1068.0
11416116,PRV55706,NaN,NaN,1.0,Y,12.0,12.0,1.0,1.0,1.0,...,2.0,1.0,1.0,20000.0,2136.0,740.0,320.0,No,12000.0,1068.0


In [125]:
final_train.shape

(11416118, 25)

In [126]:
final_test.shape

(2848907, 24)

## handling Missing Value

In [127]:
final_train.isnull().sum()

Provider                           107267
InscClaimAmtReimbursed_x           111014
DeductibleAmtPaid_x                111014
Gender                             298000
RenalDiseaseIndicator              298000
NoOfMonths_PartACov                298000
NoOfMonths_PartBCov                298000
ChronicCond_Alzheimer              298000
ChronicCond_Heartfailure           298000
ChronicCond_KidneyDisease          298000
ChronicCond_Cancer                 298000
ChronicCond_ObstrPulmonary         298000
ChronicCond_Depression             298000
ChronicCond_Diabetes               298000
ChronicCond_IschemicHeart          298000
ChronicCond_Osteoporasis           298000
ChronicCond_rheumatoidarthritis    298000
ChronicCond_stroke                 298000
IPAnnualReimbursementAmt           298000
IPAnnualDeductibleAmt              298000
OPAnnualReimbursementAmt           298000
OPAnnualDeductibleAmt              298000
PotentialFraud                     107267
InscClaimAmtReimbursed_y          

In [98]:
final_test.isnull().sum()

Provider                           124488
Gender                              61873
RenalDiseaseIndicator               61873
NoOfMonths_PartACov                 61873
NoOfMonths_PartBCov                 61873
ChronicCond_Alzheimer               61873
ChronicCond_Heartfailure            61873
ChronicCond_KidneyDisease           61873
ChronicCond_Cancer                  61873
ChronicCond_ObstrPulmonary          61873
ChronicCond_Depression              61873
ChronicCond_Diabetes                61873
ChronicCond_IschemicHeart           61873
ChronicCond_Osteoporasis            61873
ChronicCond_rheumatoidarthritis     61873
ChronicCond_stroke                  61873
IPAnnualReimbursementAmt            61873
IPAnnualDeductibleAmt               61873
OPAnnualReimbursementAmt            61873
OPAnnualDeductibleAmt               61873
Provider                           116290
InscClaimAmtReimbursed             116290
DeductibleAmtPaid                  116486
Provider                          

In [128]:
final_train.dropna(inplace=True)

In [129]:
final_test.dropna(inplace=True)

In [130]:
final_train.isnull().sum()

Provider                           0
InscClaimAmtReimbursed_x           0
DeductibleAmtPaid_x                0
Gender                             0
RenalDiseaseIndicator              0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
PotentialFraud                     0
InscClaimAmtReimbursed_y           0
DeductibleAmtPaid_y                0
dtype: int64

In [131]:
final_test.isnull().sum()

Provider                           0
InscClaimAmtReimbursed_x           0
DeductibleAmtPaid_x                0
Gender                             0
RenalDiseaseIndicator              0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
InscClaimAmtReimbursed_y           0
DeductibleAmtPaid_y                0
dtype: int64

In [132]:
final_train.shape

(10781955, 25)

In [133]:
final_test.shape

(2677401, 24)

Peneliti Memutuskan untuk tidak mengambil `miss value` untuk pembuatan model prediksi, dan terlihat perbedaan shape dikarenakan label untuk tabel test tidak ada.

## Data Preprocessing
This section contains the process of preparing data for the model training process, such as dividing data into train-dev-test, data transformation (normalization, encoding, etc.), and other processes needed.

### Get data for model inference

In [106]:
## Get data for model inference
data_inf = final_train.sample(20, random_state=30)
data_inf

,Provider,PotentialFraud,Gender,RenalDiseaseIndicator,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,...,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid
3019,PRV54773,No,2.0,0,12.0,12.0,1.0,1.0,1.0,1.0,...,0.0,0.0,50.0,10.0,PRV56123,4000.0,1068.0,PRV52628,40,70
373,PRV51471,Yes,1.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,8000.0,1068.0,1070.0,240.0,PRV51954,11000.0,1068.0,PRV51129,60,0
3315,PRV55157,No,2.0,0,12.0,12.0,1.0,1.0,2.0,1.0,...,0.0,0.0,3790.0,190.0,PRV51183,6000.0,1068.0,PRV56803,90,0
5212,PRV57525,Yes,1.0,0,12.0,12.0,1.0,1.0,2.0,1.0,...,0.0,0.0,900.0,100.0,PRV54348,6000.0,1068.0,PRV56044,40,0
1968,PRV53454,No,1.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,0.0,0.0,210.0,50.0,PRV52463,5000.0,1068.0,PRV52014,10,0
2002,PRV53496,No,2.0,0,12.0,12.0,1.0,1.0,2.0,2.0,...,0.0,0.0,1670.0,850.0,PRV52511,4000.0,1068.0,PRV52656,40,0
2807,PRV54500,No,2.0,0,12.0,12.0,1.0,2.0,2.0,2.0,...,0.0,0.0,2240.0,200.0,PRV57502,15000.0,1068.0,PRV56008,1300,0
1837,PRV53293,No,2.0,0,12.0,12.0,1.0,2.0,2.0,2.0,...,11000.0,2136.0,1170.0,650.0,PRV52173,9000.0,1068.0,PRV54688,30,0
3293,PRV55132,No,2.0,0,12.0,12.0,2.0,1.0,2.0,2.0,...,0.0,0.0,1820.0,500.0,PRV55172,8000.0,1068.0,PRV57150,40,0
1755,PRV53189,No,1.0,Y,12.0,12.0,2.0,2.0,1.0,1.0,...,12180.0,1068.0,2030.0,390.0,PRV52178,8000.0,1068.0,PRV55000,100,0


In [107]:
# remove inference set from data set
final_train = final_train.drop(data_inf.index)
final_train

,Provider,PotentialFraud,Gender,RenalDiseaseIndicator,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,...,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid
0,PRV51001,No,1.0,0,12.0,12.0,1.0,2.0,1.0,2.0,...,36000.0,3204.0,60.0,70.0,PRV55912,26000.0,1068.0,PRV56011,30,0
1,PRV51003,Yes,2.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,0.0,0.0,30.0,50.0,PRV55907,5000.0,1068.0,PRV57610,80,0
2,PRV51004,No,1.0,0,12.0,12.0,1.0,2.0,2.0,2.0,...,0.0,0.0,90.0,40.0,PRV56046,5000.0,1068.0,PRV57595,10,0
3,PRV51005,Yes,1.0,0,12.0,12.0,1.0,1.0,2.0,2.0,...,0.0,0.0,1810.0,760.0,PRV52405,5000.0,1068.0,PRV56011,40,0
4,PRV51007,No,1.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,0.0,0.0,1790.0,1200.0,PRV56614,10000.0,1068.0,PRV56011,200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,No,1.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,0.0,0.0,430.0,310.0,PRV57228,8000.0,1068.0,PRV55004,100,0
5406,PRV57760,No,1.0,0,12.0,12.0,1.0,1.0,1.0,2.0,...,0.0,0.0,400.0,210.0,PRV55048,15000.0,1068.0,PRV55004,1900,0
5407,PRV57761,No,1.0,0,12.0,12.0,2.0,1.0,2.0,2.0,...,0.0,0.0,990.0,150.0,PRV55482,11000.0,1068.0,PRV54966,20,0
5408,PRV57762,No,2.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,7400.0,2136.0,10.0,450.0,PRV56023,3000.0,1068.0,PRV55004,300,0


In [108]:
#reset index to reduce error
final_train.reset_index(drop= True, inplace=True)
data_inf.reset_index(drop=True, inplace=True)
data_inf

,Provider,PotentialFraud,Gender,RenalDiseaseIndicator,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,...,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid
0,PRV54773,No,2.0,0,12.0,12.0,1.0,1.0,1.0,1.0,...,0.0,0.0,50.0,10.0,PRV56123,4000.0,1068.0,PRV52628,40,70
1,PRV51471,Yes,1.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,8000.0,1068.0,1070.0,240.0,PRV51954,11000.0,1068.0,PRV51129,60,0
2,PRV55157,No,2.0,0,12.0,12.0,1.0,1.0,2.0,1.0,...,0.0,0.0,3790.0,190.0,PRV51183,6000.0,1068.0,PRV56803,90,0
3,PRV57525,Yes,1.0,0,12.0,12.0,1.0,1.0,2.0,1.0,...,0.0,0.0,900.0,100.0,PRV54348,6000.0,1068.0,PRV56044,40,0
4,PRV53454,No,1.0,0,12.0,12.0,2.0,2.0,2.0,2.0,...,0.0,0.0,210.0,50.0,PRV52463,5000.0,1068.0,PRV52014,10,0
5,PRV53496,No,2.0,0,12.0,12.0,1.0,1.0,2.0,2.0,...,0.0,0.0,1670.0,850.0,PRV52511,4000.0,1068.0,PRV52656,40,0
6,PRV54500,No,2.0,0,12.0,12.0,1.0,2.0,2.0,2.0,...,0.0,0.0,2240.0,200.0,PRV57502,15000.0,1068.0,PRV56008,1300,0
7,PRV53293,No,2.0,0,12.0,12.0,1.0,2.0,2.0,2.0,...,11000.0,2136.0,1170.0,650.0,PRV52173,9000.0,1068.0,PRV54688,30,0
8,PRV55132,No,2.0,0,12.0,12.0,2.0,1.0,2.0,2.0,...,0.0,0.0,1820.0,500.0,PRV55172,8000.0,1068.0,PRV57150,40,0
9,PRV53189,No,1.0,Y,12.0,12.0,2.0,2.0,1.0,1.0,...,12180.0,1068.0,2030.0,390.0,PRV52178,8000.0,1068.0,PRV55000,100,0
